In [463]:
from dash import Dash, dash_table, dcc, callback, Output, Input, clientside_callback, _dash_renderer
import pandas as pd
import plotly.express as px
import dash_mantine_components as dmc
from requests import get
from dash.dependencies import MATCH, ALL, State
from dash.exceptions import PreventUpdate
from dash import callback_context
from dash import html
import webbrowser
from threading import Timer
from dash_iconify import DashIconify
import dash_bootstrap_components as dbc


In [464]:
CSV_FILES = {
    "Category": "C:/Users/kaila/OneDrive/Desktop/COMP3610PROJECT/notebooks/dashboard/CPS_Summary.csv",
    "Brands per Category": "C:/Users/kaila/OneDrive/Desktop/COMP3610PROJECT/notebooks/dashboard/BPCPS_Summary.csv",
    "Products per Brand": "C:/Users/kaila/OneDrive/Desktop/COMP3610PROJECT/notebooks/dashboard/PPBPS_Summary.csv",
    "Products per Category": "C:/Users/kaila/OneDrive/Desktop/COMP3610PROJECT/notebooks/dashboard/PPCPS_Summary.csv"
}


In [465]:
type_map = {
        "Brands per Category": "brand",
        "Products per Category": "product",
        "Products per Brand": "product",
        "Category": "category"
    }

In [466]:
val = type_map["Brands per Category"]
val

'brand'

In [467]:
def load_data(radio_choice):
    col = type_map[radio_choice]
    df = pd.read_csv(
        CSV_FILES[radio_choice],
        usecols=[col, 'year', 'month','Popularity Score']
    )
    return df

In [468]:
def sort_by_date(df):
    return df.sort_values(by='date', ascending=True)

In [469]:
def plot_time_series(filtered_df, radio_choice, selected_val, template):

    fig = px.line(sort_by_date(filtered_df), x="date", y="Popularity Score",
                  template=template, title=f"{radio_choice} - {selected_val}")
    
    return fig

In [470]:
def plot_bar_chart(filtered_df, prod_names, selected_val, template):
    fig = px.bar(filtered_df, x="month_year", y='Popularity Score',
                 template=template, title=f"Top Products in {selected_val}:\n{', '.join(prod_names.astype(str))}")
    
    return fig

In [471]:
def get_template(theme):
    dmc.add_figure_templates()
    return "mantine_dark" if theme == "dark" else "mantine_light"

In [472]:
def get_table_styles(theme):
    if theme == "dark":
        return {
            "header": {
                "backgroundColor": "#1f2937",
                "color": "white",
                "fontWeight": "bold",
                "textAlign": "center"
            },
            "data": {
                "backgroundColor": "#111827",
                "color": "white",
                "border": "1px solid #374151"
            },
            "data_conditional": [
                {
                    "if": {"row_index": "odd"},
                    "backgroundColor": "#1f2937"
                }
            ]
        }
    else:  # light theme
        return {
            "header": {
                "backgroundColor": "#f3f4f6",
                "color": "#111827",
                "fontWeight": "bold",
                "textAlign": "center"
            },
            "data": {
                "backgroundColor": "white",
                "color": "#111827",
                "border": "1px solid #d1d5db"
            },
            "data_conditional": [
                {
                    "if": {"row_index": "odd"},
                    "backgroundColor": "#f9fafb"
                }
            ]
        }

In [ ]:

def get_data_table(df, theme):
    styles = get_table_styles(theme)

    df = df.copy()
    df['date'] = df['date'].dt.strftime('%Y-%m-%d')  # nicer date format

    table = dash_table.DataTable(
        columns=[{"name": i, "id": i} for i in df.columns],
        data=df.to_dict("records"),
        page_size=10,
        sort_action="native",
        filter_action="native",
        style_table={"overflowX": "auto"},
        style_cell={"padding": "8px", "textAlign": "left", "fontSize": "14px"},
        style_header=styles['header'],
        style_data=styles['data'],
        style_data_conditional=styles['data_conditional'],
    )

    return table

In [474]:
theme_toggle = dmc.Switch(
    offLabel=DashIconify(icon="radix-icons:sun", width=15, color=dmc.DEFAULT_THEME["colors"]["yellow"][8]),
    onLabel=DashIconify(icon="radix-icons:moon", width=15, color=dmc.DEFAULT_THEME["colors"]["yellow"][6]),
    id="color-scheme-switch",
    persistence=True,
    color="grey",
)

In [475]:
def load_brand_data(radio_choice, select_cat, select_brand, template, theme):

    brand_df = pd.read_csv(CSV_FILES[radio_choice])
    
    cat_options = [{'label': val, 'value': val} for val in sorted(brand_df['category'].unique())]

    brand_df = brand_df[brand_df['category'] == select_cat].copy()

    brand_options = [{'label': val, 'value': val} for val in brand_df['brand'].unique()]

    if not select_cat:
        return(
            cat_options, 
            html.Div('Select a category to view brand performance'),
            {'display': 'block'},
            [],
            {'display': 'block'},
            []
        )

    if not select_brand:
        return(
            cat_options,
            html.Div('Select a brand to view performance'),
            {'display': 'block'},
            brand_options,
            {'display': 'block'},
            []
        )
    
    filtered_df = brand_df[brand_df['brand'] == select_brand].copy()
    filtered_df['date'] = pd.to_datetime(filtered_df[["year", "month"]].assign(day=1))

    
    return(
        cat_options,
        dcc.Graph(figure=plot_time_series(filtered_df, radio_choice, select_brand, template)),
        {'display': 'block'}, 
        brand_options,
        {'display': 'block'},
        get_data_table(filtered_df, theme)
    )


In [ ]:
def load_prod_data(radio_choice, select_cat, select_brand, template, theme):
    
    df = load_data(radio_choice)
    filtered_df = df.head(6)

    if radio_choice == "Products per Category":
        select_year = select_brand
        options = [{'label': val, 'value': val} for val in sorted(filtered_df['category'].unique())]

        if not select_cat:
            return(
                options, 
                html.Div('Select a category to view product performance'),
                {'display':'none'},
                [],
                {'display':'block'},
                [],
            )
        
        else:

            years = [{'label': str(y), 'value': str(y)} for y in sorted(filtered_df['year'].unique())]

            if not select_year:
                return(
                    options,
                    html.Div('Select a year to view product performance'),
                    {'display':'none'},
                    years,
                    {'display':'block'},
                    [],
                )
            
    else:
        select_year = select_cat
        options = [{'label': val, 'value': val} for val in sorted(filtered_df['brand'].unique())]

        if not select_brand:
            return(
                options, 
                html.Div('Select a brand to view product performance'),
                {'display':'none'},
                [],
                {'display':'block'},
                [],
            )
        
        years = [{'label': str(y), 'value': str(y)} for y in sorted(filtered_df['year'].unique())]

        if not select_year:
            return(
                    options,
                    html.Div('Select a year to view product performance'),
                    {'display':'none'},
                    years,
                    {'display':'block'},
                )
        
    prod_names = filtered_df['product']
    filtered_df['month_year'] = filtered_df['month'].astype(str) + "-" + filtered_df['year'].astype(str)

    return(
        options, 
        dcc.Graph(figure=plot_bar_chart(filtered_df, prod_names, select_year, template)), 
        {'display': 'none'}, 
        select_year,
        {'display': 'none'},
        get_data_table(filtered_df, theme)
    )

In [477]:
def load_cat_data(radio_choice, select_cat, select_brand, template, theme):

    select_brand = []
    df = load_data(radio_choice)

    options = [{'label': val, 'value': val} for val in sorted(df['category'].unique())]

    if not select_cat:
        return(
            options, 
            html.Div('Select a category to view brand performance'),
            {'display': 'block'},
            [],
            {'display': 'none'},
            [],
        )
    
    filtered_df = df[df['category'] == select_cat].copy()
    filtered_df['date'] = pd.to_datetime(filtered_df[["year", "month"]].assign(day=1))

    return (
        options, 
        dcc.Graph(figure=plot_time_series(filtered_df, radio_choice, select_cat, template)), 
        {'display': 'block'}, 
        select_brand,
        {'display': 'none'},
        get_data_table(filtered_df, theme),
    )

In [478]:
app = Dash()

app.layout = dmc.MantineProvider(
    children=[
        theme_toggle,
        dcc.Store(id='theme-store'),
        dmc.Container([

            dmc.Title("Market Analysis Dashboard", 
                      style={"size":"h3", "textAlign": "center"}), 
        
            dmc.Grid([
                dmc.GridCol(
                    dmc.RadioGroup(
                    [dmc.Radio(i, value=i) for i in CSV_FILES.keys()],
                    id="radio-group",
                    value="Category",
                    size="md",
                    style={"marginBottom": "25px"}
                    ),
                    span=8
                ),

                dmc.GridCol(
                    dmc.Checkbox(
                    id="compare-checkbox",
                    labelPosition="right",
                    label="Compare",
                    color="#5c7cfa",
                    variant="filled",
                    size="sm",
                    radius="sm",
                    disabled=False,
                    indeterminate=False,
                    style={'display': 'block', 'justifyContent': 'center',
                        'margonBottom': '25px'},
                    ),
                    span=4,
                    style={"display": "flex", 
                           "alignItems": "top", 
                           "justifyContent": "flex-end"}
                ),
            ]),
            
            dmc.Select(
                id="first-dropdown",
                data=[],
                placeholder="Select",
                searchable=True,
                clearable=True,
                nothingFoundMessage="No matches found",
            ),

            dmc.Space(h=20),

            dmc.Select(
                id='second-dropdown',
                data=[],
                placeholder="Select",
                searchable=True,
                clearable=True,
                nothingFoundMessage="No matches found",
                style={'display': 'none'},
            ),

            dmc.Space(h=20),

            html.Div(id="plots"),
            
            dmc.Space(h=10),
            
            html.Div(id='data-table'),

        ], fluid=True,
    )],
)
@callback(
    Output("first-dropdown", "data"),
    Output("plots", "children"),
    Output("compare-checkbox", "style"),
    Output("second-dropdown", "data"),
    Output('second-dropdown', 'style'),
    Output('data-table', 'children'),
    Input("radio-group", "value"),
    Input("first-dropdown", "value"),
    Input("theme-store", "data"),
    Input('second-dropdown', 'value'),
)
def combined_update(radio_choice, select_cat, theme, select_brand):
    template = get_template(theme)

    col_map = {
        "Brands per Category": load_brand_data,
        "Products per Category": load_prod_data,
        "Products per Brand": load_prod_data,
        "Category": load_cat_data
    }

    if radio_choice in col_map:
        return col_map[radio_choice](radio_choice, select_cat, select_brand, template, theme)
    
    return [], html.Div("Invalid selection."), {'display': 'none'}, [], {'display':'block'}, []
    
clientside_callback(
    """
    (switchOn) => {
       const theme = switchOn ? 'dark' : 'light';
        document.documentElement.setAttribute('data-mantine-color-scheme', theme);
        return theme;
    }
    """,
    Output("theme-store", "data"),
    Input("color-scheme-switch", "checked"),
)


# def open_browser():
#     webbrowser.open_new("http://127.0.0.1:8050/")

if __name__ == "__main__":
    # Timer(1, open_browser).start()  # Wait 1 second then open browser
    app.run(debug=True)

[2025-08-15 18:26:39,423] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\kaila\AppData\Roaming\Python\Python313\site-packages\pandas\core\indexes\base.py", line 3812, in get_loc
    return self._engine.get_loc(casted_key)
           ~~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^
  File "pandas/_libs/index.pyx", line 167, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/index.pyx", line 196, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/hashtable_class_helper.pxi", line 7088, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas/_libs/hashtable_class_helper.pxi", line 7096, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 'category'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "C:\Users\kaila\AppData\Roaming\Python\Python313\site-packages\flask\app.py", line 917, in full_dispatch_request
    rv = self.dispatch_req